<a href="https://colab.research.google.com/github/Parth-Ach2002/Image-Classification/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing the necessary libraries**

In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

**Connecting google drive with our file in order to access the contents of the dataset**

In [2]:
# Mounting the Data from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Setting the path for all the datasets**

In [3]:
# Set path for all three datasets
train_path="/content/drive/MyDrive/Image Classification Dataset/Train"
test_path="/content/drive/MyDrive/Image Classification Dataset/Test"
val_path="/content/drive/MyDrive/Image Classification Dataset/Validation"

**Generating image data along with few transformations**

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=30, width_shift_range=10.0, height_shift_range=10.0, shear_range=10,
                                   zoom_range=0.1,
                                   vertical_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16
                                                )
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16
                                            )

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 16

                                          )

Found 210 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


**Printing the class indices**

In [6]:
training_set.class_indices

{'No Parking': 0, 'One Way': 1, 'Zebra Crossing': 2}

**Installing Keras Tuner for hyperparameter optimization**

In [7]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.0 MB/s eta 0:00:00


**Creating the model architecture**

In [8]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(hp.Int('conv1_units', min_value=32, max_value=256, step=32), (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(hp.Int('conv2_units', min_value=32, max_value=128, step=32), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Conv2D(hp.Int('conv3_units', min_value=16, max_value=64, step=16), (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('dense1_units', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(hp.Int('dense2_units', min_value=16, max_value=64, step=16), activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

We create a function for the model with "hp" as paramter. The model consists Three Convolutional layers with respective Max Pooling layers meaning the Maximum values of pixel after filtering are considered. The come the flattening layer where data is converted into 1-D array. We use softmax function to fetch the probability of all classes which sums upto 1.

The model is compiled using Adaptive movement estiamtion for escaping local minima. The loss function categorical crossentropy is used also the metrics used is accuracy.

In [9]:
from kerastuner.tuners import RandomSearch

<ipython-input-9-94471a811b41>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Importing Random search from the package kerastuner.tuners

**Perform Hyperparameter tuning**

In [11]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=8,
    directory='tuning_directory',
    project_name='my_model'
)

In [12]:
tuner.search(training_set,
             validation_data=val_set,
             epochs=30,
             batch_size=16)

Trial 8 Complete [01h 05m 38s]
val_accuracy: 0.800000011920929

Best val_accuracy So Far: 0.9666666388511658
Total elapsed time: 04h 04m 31s


In [14]:
best_hp = tuner.get_best_hyperparameters()[0]
final_model = build_model(best_hp)

In [28]:
history = final_model.fit(training_set,
                          validation_data=val_set,
                          epochs=30,
                          batch_size=16)

Epoch 1/30
14/14 [==============================] - 68s 5s/step - loss: 0.1878 - accuracy: 0.9333 - val_loss: 0.2508 - val_accuracy: 0.8667
Epoch 2/30
14/14 [==============================] - 65s 4s/step - loss: 0.1285 - accuracy: 0.9667 - val_loss: 0.3004 - val_accuracy: 0.8667
Epoch 3/30
14/14 [==============================] - 65s 4s/step - loss: 0.1062 - accuracy: 0.9714 - val_loss: 0.3385 - val_accuracy: 0.8333
Epoch 4/30
14/14 [==============================] - 62s 4s/step - loss: 0.2190 - accuracy: 0.9524 - val_loss: 0.2415 - val_accuracy: 0.9000
Epoch 5/30
14/14 [==============================] - 86s 6s/step - loss: 0.2297 - accuracy: 0.9238 - val_loss: 0.2655 - val_accuracy: 0.8667
Epoch 6/30
14/14 [==============================] - 65s 5s/step - loss: 0.1514 - accuracy: 0.9381 - val_loss: 0.1097 - val_accuracy: 0.9333
Epoch 7/30
14/14 [==============================] - 67s 5s/step - loss: 0.1280 - accuracy: 0.9619 - val_loss: 0.1657 - val_accuracy: 0.9000
Epoch 8/30
14/14 [==

In [35]:
final_model.save('model.h5')

In [32]:
(loss, accuracy) = final_model.evaluate(test_set)
print(loss)
print(accuracy)

4/4 [==============================] - 4s 990ms/step - loss: 0.0640 - accuracy: 0.9833
0.06399316340684891
0.9833333492279053


In [42]:
pip install gradio

In [43]:
import gradio as gr
from PIL import Image
import numpy as np

def predict_image(image):
    # Preprocess the image (resize, normalize)
    image = Image.fromarray(image)
    image = image.resize((224, 224))
    image = np.array(image) / 255.0

    # Make predictions using the model
    predictions = final_model.predict(np.expand_dims(image, axis=0))
    predicted_class = class_names[np.argmax(predictions)]

    return predicted_class

In [44]:
image_input = gr.inputs.Image(shape=(224, 224))
label_output = gr.outputs.Label()

gr1=gr.Interface(fn=predict_image, inputs=image_input, outputs=label_output).launch(share=True)

<ipython-input-44-07dc849683ea>:1: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image_input = gr.inputs.Image(shape=(224, 224))
<ipython-input-44-07dc849683ea>:1: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image_input = gr.inputs.Image(shape=(224, 224))
<ipython-input-44-07dc849683ea>:2: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label_output = gr.outputs.Label()
<ipython-input-44-07dc849683ea>:2: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label_output = gr.outputs.Label()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f90e58808857b7cacd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
